In [1]:
import pandas as pd

# parquet 파일 경로 지정
file_path = 'singleq.parquet'

# pyarrow(또는 fastparquet) 기반으로 parquet 파일 읽기
df = pd.read_parquet(file_path)

In [2]:
df.head()

,cleaning_status,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_strategy,input,output_program,output_answer,split,dataset
0,consensus,Solve the following math word problem.\n\nJohn...,Solve the following math word problem.\n\nJohn...,[48],[48],math,John can read 4 books a day. John reads every ...,"from sympy import symbols, solve\nx = symbols(...",48,test,singleq
1,consensus,Solve the following math word problem.\n\nSam ...,Solve the following math word problem.\n\nSam ...,[1],[1],math,"Sam grew 4 watermelons, but the rabbits ate 3 ...","from sympy import symbols, solve\nx = symbols(...",1,test,singleq
2,consensus,Solve the following math word problem.\n\nTo f...,Solve the following math word problem.\n\nTo f...,[111421],[111421],math,"To fill an order, the factory dyed 61,921 yard...","from sympy import symbols, solve\nx = symbols(...",111421,test,singleq
3,consensus,Solve the following math word problem.\n\nMari...,Solve the following math word problem.\n\nMari...,[9],[9],math,Maria needs 21 cartons of berries to make a be...,"from sympy import symbols, solve\nx = symbols(...",9,test,singleq
4,consensus,Solve the following math word problem.\n\nBett...,Solve the following math word problem.\n\nBett...,[11],[11],math,Betty bought 88 pink flower stones and wanted ...,"from sympy import symbols, solve\nx = symbols(...",11,test,singleq


In [3]:
df['platinum_prompt'][0]

'Solve the following math word problem.\n\nJohn can read 4 books a day. John reads every Monday and Tuesday. How many books would John read in 6 weeks?\n\nThink step-by-step. Then, provide the final answer as a single number in the format "Answer: XXX" with no extra formatting.'

In [4]:
import pandas as pd
import re

# 1. Read the question again: ...
def repeat_question(match):
    # match.group(1): 첫 번째 줄바꿈 (예: "\n\n" 또는 "\n   \n")
    # match.group(2): 질문 텍스트
    # match.group(3): 두 번째 줄바꿈 (예: "\n\n")
    # 원하는 결과: 첫 번째 줄바꿈 뒤 기존 질문 대신 "Read the question again:" + 질문 텍스트를 두 번 반복하고,
    # 두 번째 줄바꿈을 그대로 붙인다.
    return f"{match.group(1)}{match.group(2)}\n\nRead the question again: {match.group(2)}{match.group(3)}"

# \n\n 사이에 공백이 있을 수도 있으므로 \n\s*\n 형태로 패턴을 수정
df['RE2'] = df['platinum_prompt'].str.replace(
    r'(\n\s*\n)(.*?)(\n\s*\n)(?=Think step-by-step)',
    repeat_question,
    flags=re.DOTALL,
    regex=True
)

# 2. Summarize the question first.
df['sum'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Summarize the question first. ",
    regex=True
)

# 3. Organize the question into a table first.
df['table'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Organize the question into a table first. ",
    regex=True
)

# 4. Organize the question into a graph structure first.
df['graph'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Organize the question into a graph structure first. ",
    regex=True
)

# 5. Summarize the question in bullet points first.
df['bullet_point'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Summarize the question in bullet points first. ",
    regex=True
)

# 6. Read the question again first.
df['sRE2'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Read the question again first. ",
    regex=True
)

In [10]:
df['graph'][0]

'Solve the following math word problem.\n\nJohn can read 4 books a day. John reads every Monday and Tuesday. How many books would John read in 6 weeks?\n\nOrganize the question into a graph structure first. Think step-by-step. Then, provide the final answer as a single number in the format "Answer: XXX" with no extra formatting.'

In [11]:
df['platinum_prompt_no_cot'][0]

'Solve the following math word problem.\n\nJohn can read 4 books a day. John reads every Monday and Tuesday. How many books would John read in 6 weeks?\n\nThen, provide the final answer as a single number in the format "Answer: XXX" with no extra formatting.'

In [12]:
import pandas as pd
import re

def repeat_question(match):
    # match.group(1): 첫 번째 줄바꿈 (예: "\n\n" 또는 "\n   \n")
    # match.group(2): 질문 텍스트
    # match.group(3): 두 번째 줄바꿈 (예: "\n\n")
    # 원하는 결과: 첫 번째 줄바꿈 뒤 기존 질문 대신 "Read the question again:" + 질문 텍스트를 두 번 반복하고,
    # 두 번째 줄바꿈을 그대로 붙인다.
    return f"{match.group(1)}{match.group(2)}\n\nRead the question again: {match.group(2)}{match.group(3)}"

# \n\n 사이에 공백이 있을 수도 있으므로 \n\s*\n 형태로 패턴을 수정
df['RE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(\n\s*\n)(.*?)(\n\s*\n)(?=Then, provide the final answer as a single number)',
    repeat_question,
    flags=re.DOTALL,
    regex=True
)

df['sum_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer as a single number)',
    "Summarize the question first. ",
    regex=True
)

df['table_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer as a single number)',
    "Organize the question into a table first. ",
    regex=True
)

df['graph_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer as a single number)',
    "Organize the question into a graph structure first. ",
    regex=True
)

df['bullet_point_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer as a single number)',
    "Summarize the question in bullet points first. ",
    regex=True
)

df['sRE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer as a single number)',
    "Read the question again first. ",
    regex=True
)

In [20]:
df['bullet_point_no_cot'][0]

'Solve the following math word problem.\n\nJohn can read 4 books a day. John reads every Monday and Tuesday. How many books would John read in 6 weeks?\n\nSummarize the question in bullet points first. Then, provide the final answer as a single number in the format "Answer: XXX" with no extra formatting.'

In [21]:
df.to_parquet('singleq_add_task.parquet')

In [23]:
df.head()

,cleaning_status,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_strategy,input,output_program,output_answer,split,...,table,graph,bullet_point,sRE2,RE2_no_cot,sum_no_cot,table_no_cot,graph_no_cot,bullet_point_no_cot,sRE2_no_cot
0,consensus,Solve the following math word problem.\n\nJohn...,Solve the following math word problem.\n\nJohn...,[48],[48],math,John can read 4 books a day. John reads every ...,"from sympy import symbols, solve\nx = symbols(...",48,test,...,Solve the following math word problem.\n\nJohn...,Solve the following math word problem.\n\nJohn...,Solve the following math word problem.\n\nJohn...,Solve the following math word problem.\n\nJohn...,Solve the following math word problem.\n\nJohn...,Solve the following math word problem.\n\nJohn...,Solve the following math word problem.\n\nJohn...,Solve the following math word problem.\n\nJohn...,Solve the following math word problem.\n\nJohn...,Solve the following math word problem.\n\nJohn...
1,consensus,Solve the following math word problem.\n\nSam ...,Solve the following math word problem.\n\nSam ...,[1],[1],math,"Sam grew 4 watermelons, but the rabbits ate 3 ...","from sympy import symbols, solve\nx = symbols(...",1,test,...,Solve the following math word problem.\n\nSam ...,Solve the following math word problem.\n\nSam ...,Solve the following math word problem.\n\nSam ...,Solve the following math word problem.\n\nSam ...,Solve the following math word problem.\n\nSam ...,Solve the following math word problem.\n\nSam ...,Solve the following math word problem.\n\nSam ...,Solve the following math word problem.\n\nSam ...,Solve the following math word problem.\n\nSam ...,Solve the following math word problem.\n\nSam ...
2,consensus,Solve the following math word problem.\n\nTo f...,Solve the following math word problem.\n\nTo f...,[111421],[111421],math,"To fill an order, the factory dyed 61,921 yard...","from sympy import symbols, solve\nx = symbols(...",111421,test,...,Solve the following math word problem.\n\nTo f...,Solve the following math word problem.\n\nTo f...,Solve the following math word problem.\n\nTo f...,Solve the following math word problem.\n\nTo f...,Solve the following math word problem.\n\nTo f...,Solve the following math word problem.\n\nTo f...,Solve the following math word problem.\n\nTo f...,Solve the following math word problem.\n\nTo f...,Solve the following math word problem.\n\nTo f...,Solve the following math word problem.\n\nTo f...
3,consensus,Solve the following math word problem.\n\nMari...,Solve the following math word problem.\n\nMari...,[9],[9],math,Maria needs 21 cartons of berries to make a be...,"from sympy import symbols, solve\nx = symbols(...",9,test,...,Solve the following math word problem.\n\nMari...,Solve the following math word problem.\n\nMari...,Solve the following math word problem.\n\nMari...,Solve the following math word problem.\n\nMari...,Solve the following math word problem.\n\nMari...,Solve the following math word problem.\n\nMari...,Solve the following math word problem.\n\nMari...,Solve the following math word problem.\n\nMari...,Solve the following math word problem.\n\nMari...,Solve the following math word problem.\n\nMari...
4,consensus,Solve the following math word problem.\n\nBett...,Solve the following math word problem.\n\nBett...,[11],[11],math,Betty bought 88 pink flower stones and wanted ...,"from sympy import symbols, solve\nx = symbols(...",11,test,...,Solve the following math word problem.\n\nBett...,Solve the following math word problem.\n\nBett...,Solve the following math word problem.\n\nBett...,Solve the following math word problem.\n\nBett...,Solve the following math word problem.\n\nBett...,Solve the following math word problem.\n\nBett...,Solve the following math word problem.\n\nBett...,Solve the following math word problem.\n\nBett...,Solve the following math word problem.\n\nBett...,Solve the following math word problem.\n\nBett...
